<a href="https://colab.research.google.com/github/Ken-huea/Language-processing/blob/main/%E6%A5%B5%E6%80%A7%E8%BE%9E%E6%9B%B8(Mecab)_%E6%B8%85%E6%9B%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###### 資料召喚コード

### 概要や今回のテーマ、イントロ等
[ネガポジ](https://premium.aidemy.jp/courses/6020)

自然言語処理を用いて、文章などに含まれる評価・感情に関する表現を<font color="red">ポジティブ/ネガティブ</font>かを分析する。

感情分析（Sentiment Analysis）と呼ばれる技術の一種。

ネガ・ポジ分析の手法として、極性辞書を使った単語ごとの分類や、ディープラーニングを使ったものもあります。

今回は極性辞書を用いてネガポジ分析を行い、文章のネガポジポイントの平均値を算出し、全体的な総評をしてみたいと思います。

その後はポジティブそうな歌詞を読み込ませて、どのくらいポジティブなのか確認したいと思います。

## 実装

### データの読込と形態素解析
[Aidemyの教材はこちら](https://premium.aidemy.jp/courses/6020/exercises/KnmTeGK_DM)

形態素解析の準備。　質問は
[こちら](https://teratail.com/questions/110364)
　[こちら](https://aidemy-communication.slack.com/archives/C03ANT80SAE/p1662181545790699)

In [ ]:
# メカブのインポート
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

In [ ]:
import MeCab

データの読込。適当なテキストファイルを読み込みます。

In [ ]:
mecab = MeCab.Tagger('')

title = open('./6020_negative_positive_data/data/aidokushono_insho.txt')
file = title.read()
title.close()

print(type(file))           # str は形態素解析できるけど、listは出来ない？
# print(mecab.parse(file))
file

<class 'str'>


'愛読書の印象\n芥川龍之介\n\n-------------------------------------------------------\n【テキスト中に現れる記号について】\n\n《》：ルビ\n（例）諳記《あんき》してゐた\n\n［＃］：入力者注\u3000主に外字の説明や、傍点の位置の指定\n（例）又ため［＃「ため」に傍点］にもなる本である\n-------------------------------------------------------\n\n\u3000子供の時の愛読書は「西遊記」が第一である。これ等は今日でも僕の愛読書である。比喩談としてこれほどの傑作は、西洋には一つもないであらうと思ふ。名高いバンヤンの「天路歴程」なども到底この「西遊記」の敵ではない。それから「水滸伝」も愛読書の一つである。これも今以て愛読してゐる。一時は「水滸伝」の中の一百八人の豪傑の名前を悉く諳記《あんき》してゐたことがある。その時分でも押川春浪氏の冒険小説や何かよりもこの「水滸伝」だの「西遊記」だのといふ方が遥かに僕に面白かつた。\n\u3000中学へ入学前から徳富蘆花氏の「自然と人生」や樗牛の「平家雑感」や小島烏水氏の「日本山水論」を愛読した。同時に、夏目さんの「猫」や鏡花氏の「風流線」や緑雨の「あられ酒」を愛読した。だから人の事は笑へない。僕にも「文章倶楽部」の「青年文士録」の中にあるやうな「トルストイ、坪内士行、大町桂月」時代があつた。\n\u3000中学を卒業してから色んな本を読んだけれども、特に愛読した本といふものはないが、概して云ふと、ワイルドとかゴーチエとかいふやうな絢爛《けんらん》とした小説が好きであつた。それは僕の気質からも来てゐるであらうけれども、一つは慥《たし》かに日本の自然主義的な小説に厭きた反動であらうと思ふ。ところが、高等学校を卒業する前後から、どういふものか趣味や物の見方に大きな曲折が起つて、前に言つたワイルドとかゴーチエとかといふ作家のものがひどくいやになつた。ストリンドベルクなどに傾倒したのはこの頃である。その時分の僕の心持からいふと、ミケエロ・アンヂエロ風な力を持つてゐない芸術はすべて瓦礫のやうに感じられた。これは当時読んだ「ジヤンクリストフ」などの影響であつたらうと思ふ。\n\u3000さういふ心持が大学を卒業する後まで

こちらの独自関数にかけて、形態素解析結果を辞書&リスト型にしています。独自関数の機能は

- 分かち書きを行う。
- 邪魔な要素を削除する
- 分かち書きの結果を辞書型にして格納する

こちらAidemyの教材(形態素解析)を私なりにアレンジしたものです。形態素解析しても、分かち書きの部分しか用いないならば

こっちの方が無駄がなく、
可読性も高いと思いますが、いかがでしょうか？


In [ ]:
import re
mecab = MeCab.Tagger('-Owakati')                     # 分かち書きに指定
def get_diclist(file):
    parsed = mecab.parse(file)                       # 分かち書きを行います。
    # 解析結果を改行ごとに区切ります
    lines = parsed.split(' ')                        # 文章を" " 空行で区切っています。
    lines = lines[:-2]                               # これが機能していない？
    #解析結果のリストを作成します
    diclist = []                                    # 解析結果はここに格納されます。
    for word in lines:
        datalist = {'BaseForm':word}                # 分かち書き結果をpick upして"BaseForm"というkey名を付けてvalueとして格納。
        diclist.append(datalist)                    # 追加
    return(diclist)

# こちらAidemyの教材(形態素解析)を私なりにアレンジしたものです。形態素解析しても、分かち書きの部分しか用いないならばこっちの方が無駄がなく、可読性も高いと思いますが、いかがでしょうか？

In [ ]:
# このfileはstr 返り値はlist　Aidemyの教材と結果は同じ。 
wordlist = get_diclist(file)
print(wordlist)

[{'BaseForm': '愛読'}, {'BaseForm': '書'}, {'BaseForm': 'の'}, {'BaseForm': '印象'}, {'BaseForm': '芥川'}, {'BaseForm': '龍之介'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-'}, {'BaseForm': '-------------------------'}, {'BaseForm': '【'}, {'BaseForm': 'テキスト'}, {'BaseForm': '中'}, {'BaseForm': 'に'}, {'BaseForm': '現れる'}, {'BaseForm': '記号'}, {'BaseForm': 'について'}, {'BaseForm': '】'}, {'BaseForm': '《'}, {'BaseForm': '》'}, {'BaseForm': '：'}, {'BaseForm': 'ルビ'}, {'BaseForm': '（'}, {'BaseForm

### ネガポジ判定
辞書と形態素解析済みの文章(typeリスト)を用いて判定を行いましょう

###### 極性辞書作成

ネガ/ポジのことを「極性」と呼ぶ。⇒極性辞書 = ネガポジ辞書
　
名前の通り単語ごとにネガポジ属性を付与してまとめたもの。　[こちら](https://premium.aidemy.jp/courses/6020/exercises/7UB9y2_hQh)

辞書の確認と調整

In [ ]:
# ネガポジ辞書の召喚。
import pandas as pd

# カラムも分かりやすく設定して､
pn_df = pd.read_csv('./6020_negative_positive_data/data/pn_ja.dic',sep=':',encoding='utf-8',names=('単語','読み方','品詞', 'PN') )

# ネガポジ辞書の表示。　こんな感じで　１：positive -1：negative　で数値化しているのですね。 
pn_df.head(2)

,単語,読み方,品詞,PN
0,優れる,すぐれる,動詞,1.000000
1,良い,よい,形容詞,0.999995


In [ ]:
# 今回使わないカラムを削除する
df_pn = pn_df.drop(columns = ["読み方","品詞"])
df_pn.head(2)
# こっちの方が見やすい。

,単語,PN
0,優れる,1.000000
1,良い,0.999995


pandasから上手に<font color="red">辞書型 & リスト</font>にしています。keyとvalueを設定して。

In [ ]:
# PNTableを単語と極性値のみのdict型に変更します ⇒　①辞書型に変更　key:word ,value:PN で。 

key     = df_pn['単語']            # df_pn のカラム['単語']をkeyとして設定
value   = df_pn['PN']              # df_pn のカラム['PN']をvalueとして設定
PN_dict = dict(zip(key, value))    # 上記の材料で辞書を作成
PN_dict

# これで辞書型の極性辞書が出来ました。

###### 文章と極性辞書を掛け合わせる。
質問：見慣れない表現有。

In [ ]:
# 先ほど作成した極性辞書の確認
PN_dict

In [ ]:
# 分かち書きされた文章、のおさらい。
print(wordlist)

[{'BaseForm': '愛読', 'PN': -0.207109}, {'BaseForm': '書', 'PN': -0.688965}, {'BaseForm': 'の', 'PN': 'not found'}, {'BaseForm': '印象', 'PN': -0.370506}, {'BaseForm': '芥川', 'PN': 'not found'}, {'BaseForm': '龍之介', 'PN': 'not found'}, {'BaseForm': '-', 'PN': 'not found'}, {'BaseForm': '-', 'PN': 'not found'}, {'BaseForm': '-', 'PN': 'not found'}, {'BaseForm': '-', 'PN': 'not found'}, {'BaseForm': '-', 'PN': 'not found'}, {'BaseForm': '-', 'PN': 'not found'}, {'BaseForm': '-', 'PN': 'not found'}, {'BaseForm': '-', 'PN': 'not found'}, {'BaseForm': '-', 'PN': 'not found'}, {'BaseForm': '-', 'PN': 'not found'}, {'BaseForm': '-', 'PN': 'not found'}, {'BaseForm': '-', 'PN': 'not found'}, {'BaseForm': '-', 'PN': 'not found'}, {'BaseForm': '-', 'PN': 'not found'}, {'BaseForm': '-', 'PN': 'not found'}, {'BaseForm': '-', 'PN': 'not found'}, {'BaseForm': '-', 'PN': 'not found'}, {'BaseForm': '-', 'PN': 'not found'}, {'BaseForm': '-', 'PN': 'not found'}, {'BaseForm': '-', 'PN': 'not found'}, {'BaseForm':



```
# ここで辞書型の復習
pn_list[0]              # {'BaseForm': '愛読', 'PN': -0.207109}
pn_list[0]["BaseForm"]  # "愛読"     valueの抽出
pn_list[0]["PN"]        # -0.207109  valueの抽出
```



これで辞書と形態素解析した単語達が揃いました。双方を掛け合わせましょう。　
<font color="red">Aidemyの教材コード</font>から大分いじくっています。

In [ ]:
# 解析結果のリストからPNTableに存在する単語を抽出します

def add_pnvalue(old_wordlist):
    diclist_new = []
    new_pn_list = []                                                   # 入れ物の作成
    for word in old_wordlist:
        baseword = word['BaseForm']                                    # old_wordlistの分かち書きされた単語達。
        if baseword in PN_dict:                                        # in PN_dic⇒PN dict のkey 
            # PNTableに存在していれば極性値とその単語を追加します ⇒ floatにしている。 in dict python 
            pn = float(PN_dict[baseword])
        else:
            # 存在していなければnotfoundを明記します
            pn = 'not found'
        word['PN'] = pn                                                # 独自関数により新しく篩い分けられたpnを新たなPNのvalueとします。
        diclist_new.append(word)
    for d in diclist_new:                                              # ここのfor文で not found を間引いています。⇒ちょっと独自関数に無駄がないかな？
      if d['PN'] != 'not found':
        new_pn_list.append(d)   
    
    df = pd.DataFrame(new_pn_list)                                     # DFに変換しています。
    return(df)

上の独自関数のポイントは`if baseword in PN_dict:`です。これは良くある`for i in list1:`とは<font color="red">異なる機能を持つin</font>です。

`if baseword in PN_dict:`　

PN_dictのkeyとbasewordが<font color="red">一致している</font>時、という意味です。その後にif文につなぎます。
　　
`for a in b`かと思って手間取ったので。

それとも、basewordの要素がPN_dictのkeyに一つでも入っていたら？という事？



In [ ]:
Result_PN_Akudagawa = add_pnvalue(wordlist) # 独自関数に噛ませた結果を確認してみましょう。
Result_PN_Akudagawa

,BaseForm,PN
0,愛読,-0.207109
1,書,-0.688965
2,印象,-0.370506
3,テキスト,-0.315420
4,中,-0.979874
...,...,...
227,青空,-0.010875
228,文庫,-0.536820
229,入力,-0.273497
230,校正,-0.468858


### 判定が終わった後の総評価

In [ ]:
# 極性辞書に噛ませた文章のPNに着目してみましょう。
Result_PN_Akudagawa["PN"].describe()

count    232.000000
mean      -0.492728
std        0.431765
min       -0.999997
25%       -0.885089
50%       -0.512985
75%       -0.273497
max        0.997259
Name: PN, dtype: float64

`mean      -0.492728`

結論：テスト用に読み込んだこの文章は結構なネガティブ寄りだ。芥川だから予想はしてたけど。

### 別の文章を読み込んでみる

とりあえず適当なポジティブそうな歌詞を読み込ませて極性辞書のネガポジ判定をさせてみる。　例えば･･･

#### 長渕剛の｢乾杯｣

こんなのポジティブに決まっている。｢君に幸あれ｣って言ってますからね。

In [ ]:
Nagabuchi = '''かたい絆に 思いをよせて
            語り尽くせぬ 青春の日々
            時には傷つき 時には喜び
            肩をたたきあった あの日
            あれから どれくらいたったのだろう
            沈む夕陽を いくつ数えたろう
            故郷の友は 今でも君の 心の中にいますか
            
            乾杯!今君は人生の
            大きな 大きな 舞台に立ち
            遥か長い道のりを 歩き始めた
            君に幸せあれ!

            キャンドルライトの中の二人を
            今こうして 目を細めてる
            大きな喜びと 少しのさみしさを
            涙の言葉で歌いたい
            明日の光を 身体にあびて
            ふり返らずに そのまま行けばよい
            風に吹かれても 雨に打たれても
            信じた愛に 背を向けるな

            乾杯!今君は人生の
            大きな 大きな 舞台に立ち
            遥か長い道のりを 歩き始めた
            君に幸せあれ!

            乾杯!今君は人生の
            大きな 大きな 舞台に立ち
            遥か長い道のりを 歩き始めた
            君に幸せあれ!
              '''

記事を読み込んだ際に使用した独自関数は最後の2行を削除しているので、この歌詞には使用できません。なので別の関数を用意しました。

In [ ]:
import re
mecab = MeCab.Tagger('-Owakati')                     # 分かち書きに指定
def get_diclist2(file):
    parsed = mecab.parse(file)                       # 分かち書きを行います。
    # 解析結果を改行ごとに区切ります
    lines = parsed.split(' ')                        # 文章を" " 空行で区切っています。
                                                     # 後ろ2行を削除した新しいリストを作ります
    #解析結果のリストを作成します
    diclist = []                                    # 解析結果はここに格納されます。
    for word in lines:
        datalist = {'BaseForm':word}                # 分かち書き結果をpick upして"BaseForm"というkey名を付けてvalueとして格納。
        diclist.append(datalist)                    # 追加
    return(diclist)

In [ ]:
# 無事分かち書き出来ました。
Nagabuchi2 = get_diclist2(Nagabuchi)
print(Nagabuchi2)

[{'BaseForm': 'かたい'}, {'BaseForm': '絆'}, {'BaseForm': 'に'}, {'BaseForm': '思い'}, {'BaseForm': 'を'}, {'BaseForm': 'よせ'}, {'BaseForm': 'て'}, {'BaseForm': '語り'}, {'BaseForm': '尽くせ'}, {'BaseForm': 'ぬ'}, {'BaseForm': '青春'}, {'BaseForm': 'の'}, {'BaseForm': '日々'}, {'BaseForm': '時'}, {'BaseForm': 'に'}, {'BaseForm': 'は'}, {'BaseForm': '傷つき'}, {'BaseForm': '時'}, {'BaseForm': 'に'}, {'BaseForm': 'は'}, {'BaseForm': '喜び'}, {'BaseForm': '肩'}, {'BaseForm': 'を'}, {'BaseForm': 'たたき'}, {'BaseForm': 'あっ'}, {'BaseForm': 'た'}, {'BaseForm': 'あの'}, {'BaseForm': '日'}, {'BaseForm': 'あれ'}, {'BaseForm': 'から'}, {'BaseForm': 'どれ'}, {'BaseForm': 'くらい'}, {'BaseForm': 'たっ'}, {'BaseForm': 'た'}, {'BaseForm': 'の'}, {'BaseForm': 'だろ'}, {'BaseForm': 'う'}, {'BaseForm': '沈む'}, {'BaseForm': '夕陽'}, {'BaseForm': 'を'}, {'BaseForm': 'いくつ'}, {'BaseForm': '数え'}, {'BaseForm': 'たろ'}, {'BaseForm': 'う'}, {'BaseForm': '故郷'}, {'BaseForm': 'の'}, {'BaseForm': '友'}, {'BaseForm': 'は'}, {'BaseForm': '今'}, {'BaseForm': 'でも'}, {'BaseForm': '君'},

そうしましたら、それを先ほど作成した`add_pnvalue`関数にかけます。
　これでnot foundを除いたPNリストがdfにて作成されるはずです。

In [ ]:
Result_PN_Nagabuchi= add_pnvalue(Nagabuchi2) # 独自関数に噛ませた結果を確認してみましょう。
Result_PN_Nagabuchi

,BaseForm,PN
0,絆,-0.325200
1,思い,-0.678658
2,ぬ,-0.998545
3,青春,0.971646
4,時,-0.753339
5,時,-0.753339
6,喜び,0.998861
7,肩,-0.908549
8,日,-0.903573
9,沈む,-0.907695


In [ ]:
Result_PN_Nagabuchi["PN"].describe()

count    47.000000
mean     -0.470664
std       0.661612
min      -0.998545
25%      -0.908122
50%      -0.803359
75%      -0.283165
max       0.998861
Name: PN, dtype: float64

結果は<font color="red">mean     -0.470664</font>でした。

思ってたよりもずっとネガティブでした。　文脈や雰囲気からしてかなりポジティブな曲なのですが。当然歌詞も。

なんででしょうか・・・？

個人的にはdfの中に格納されているポジティブな可視が軒並みネガティブ判定されている気がする。
サビの｢今、人生、舞台、遥か｣これが全部マイナス判定ですか・この辞書は。

#### 木村カエラの｢Butterfly｣


結婚式の定番ソングです。これならポジティブ判定になるでしょう。
私これ好きじゃないですけどね。コメント欄覗くとえずきますよ。

In [ ]:
Kaera = '''
    Butterfly　今日は今までの　どんな時より　素晴らしい
    赤い糸でむすばれてく　光の輪のなかへ

    Butterfly　今日は今までの　どんな君より　美しい
    白い羽ではばたいてく　幸せと共に

    思い出してるよ　君と出会ったころ
    何度もくり返した季節は
    二人を変えてきたね

    君は今誓い　愛する人の側で
    幸せだよと　微笑んでる
    確かなその思いで　鐘が響くよ

    太陽は沈み　いたずらに星は昇る
    夜は眠り　朝を待つ

    Butterfly　今日は今までの　どんな時より　素晴らしい
    赤い糸でむすばれてく　光の輪のなかへ

    優しさにあふれた　君がとても大好き
    悲しみあれば　共に泣いて　喜びがあるなら　共に笑うよ

    たったひとつだけ　暖かい愛に包まれ
    夢の全ては　いつまでも　つづくよ

    Butterfly　今日は今までの　どんな君より　美しい
    白い羽ではばたいてく　幸せと共に

    Butterfly　今日は今までの　どんな時より　素晴らしい
    赤い糸でむすばれてく　光の輪のなかへ

    運命の花を見つけた　チョウは青い空を舞う
    '''

In [ ]:
Kaera_wachi = get_diclist2(Kaera)
print(Kaera_wachi)

# なんかよく分からない[\u3000]とか入ってますけど,not foundで弾かれると思うので無視しても良いでしょう。

[{'BaseForm': 'Butterfly'}, {'BaseForm': '\u3000'}, {'BaseForm': '今日'}, {'BaseForm': 'は'}, {'BaseForm': '今'}, {'BaseForm': 'まで'}, {'BaseForm': 'の'}, {'BaseForm': '\u3000'}, {'BaseForm': 'どんな'}, {'BaseForm': '時'}, {'BaseForm': 'より'}, {'BaseForm': '\u3000'}, {'BaseForm': '素晴らしい'}, {'BaseForm': '赤い'}, {'BaseForm': '糸'}, {'BaseForm': 'で'}, {'BaseForm': 'むすば'}, {'BaseForm': 'れ'}, {'BaseForm': 'て'}, {'BaseForm': 'く'}, {'BaseForm': '\u3000'}, {'BaseForm': '光'}, {'BaseForm': 'の'}, {'BaseForm': '輪'}, {'BaseForm': 'の'}, {'BaseForm': 'なか'}, {'BaseForm': 'へ'}, {'BaseForm': 'Butterfly'}, {'BaseForm': '\u3000'}, {'BaseForm': '今日'}, {'BaseForm': 'は'}, {'BaseForm': '今'}, {'BaseForm': 'まで'}, {'BaseForm': 'の'}, {'BaseForm': '\u3000'}, {'BaseForm': 'どんな'}, {'BaseForm': '君'}, {'BaseForm': 'より'}, {'BaseForm': '\u3000'}, {'BaseForm': '美しい'}, {'BaseForm': '白い'}, {'BaseForm': '羽'}, {'BaseForm': 'で'}, {'BaseForm': 'はばたい'}, {'BaseForm': 'て'}, {'BaseForm': 'く'}, {'BaseForm': '\u3000'}, {'BaseForm': '幸せ'}, {'Base

In [ ]:
Result_PN_Kaera = add_pnvalue(Kaera_wachi) # 独自関数に噛ませた結果を確認してみましょう。
Result_PN_Kaera

# あれっ？なんか物凄くマイナスが目立つのですが大丈夫でしょうか...？

,BaseForm,PN
0,今日,0.172375
1,今,-0.851644
2,時,-0.753339
3,素晴らしい,0.998617
4,赤い,-0.926588
...,...,...
62,輪,-0.912371
63,運命,-0.562723
64,青い,-0.724730
65,空,-0.900915


In [ ]:
Result_PN_Kaera["PN"].describe()

count    67.000000
mean     -0.482106
std       0.631255
min      -0.999241
25%      -0.897571
50%      -0.790421
75%      -0.269392
max       0.998861
Name: PN, dtype: float64

結果物凄くネガティブになってしまいました。なんで...。

#### オフコースの「さよなら」

どんなポジティブな曲もネガティブ判定されてしまうので今度は思いきりネガティブな曲を入れたいと思います。

本当はradioheadをぶち込みたいのですが、今回も邦楽で…

In [ ]:
Sayonara = '''
    もう 終わりだね 君が小さく見える
    僕は思わず君を 抱きしめたくなる
    「私は泣かないから このままひとりにして」
    君のほほを涙が 流れては落ちる
    「僕らは自由だね」いつかそう話したね
    まるで今日のことなんて 思いもしないで
    さよなら さよなら さよなら
    もうすぐ外は白い冬
    愛したのはたしかに君だけ
    そのままの君だけ

    愛は哀しいね 僕のかわりに君が
    今日は誰かの胸に 眠るかも知れない
    僕がてれるから 誰も見ていない道を
    寄りそい歩ける寒い日が 君は好きだった
    さよなら さよなら さよなら
    もうすぐ外は白い冬
    愛したのはたしかに君だけ
    そのままの君だけ

    さよなら さよなら さよなら
    もうすぐ外は白い冬
    愛したのはたしかに君だけ
    そのままの君だけ

    さよなら さよなら さよなら
    もうすぐ外は白い冬
    愛したのはたしかに君だけ
    そのままの君だけ

    外は今日も雨 やがて雪になって
    僕らの心のなかに 降り積るだろう
    降り積るだろう
'''

In [ ]:
Sayonara_wachi = get_diclist2(Sayonara)
print(Sayonara_wachi)

# なんかよく分からない[\u3000]とか入ってますけど,not foundで弾かれると思うので無視しても良いでしょう。

[{'BaseForm': 'もう'}, {'BaseForm': '終わり'}, {'BaseForm': 'だ'}, {'BaseForm': 'ね'}, {'BaseForm': '君'}, {'BaseForm': 'が'}, {'BaseForm': '小さく'}, {'BaseForm': '見える'}, {'BaseForm': '僕'}, {'BaseForm': 'は'}, {'BaseForm': '思わず'}, {'BaseForm': '君'}, {'BaseForm': 'を'}, {'BaseForm': '抱きしめ'}, {'BaseForm': 'たく'}, {'BaseForm': 'なる'}, {'BaseForm': '「'}, {'BaseForm': '私'}, {'BaseForm': 'は'}, {'BaseForm': '泣か'}, {'BaseForm': 'ない'}, {'BaseForm': 'から'}, {'BaseForm': 'この'}, {'BaseForm': 'まま'}, {'BaseForm': 'ひとり'}, {'BaseForm': 'に'}, {'BaseForm': 'し'}, {'BaseForm': 'て'}, {'BaseForm': '」'}, {'BaseForm': '君'}, {'BaseForm': 'の'}, {'BaseForm': 'ほほ'}, {'BaseForm': 'を'}, {'BaseForm': '涙'}, {'BaseForm': 'が'}, {'BaseForm': '流れ'}, {'BaseForm': 'て'}, {'BaseForm': 'は'}, {'BaseForm': '落ちる'}, {'BaseForm': '「'}, {'BaseForm': '僕ら'}, {'BaseForm': 'は'}, {'BaseForm': '自由'}, {'BaseForm': 'だ'}, {'BaseForm': 'ね'}, {'BaseForm': '」'}, {'BaseForm': 'いつか'}, {'BaseForm': 'そう'}, {'BaseForm': '話し'}, {'BaseForm': 'た'}, {'BaseForm': 'ね'},

In [ ]:
Result_PN_Sayonara = add_pnvalue(Sayonara_wachi)
Result_PN_Sayonara
# ほぼほぼマイナスですね。

In [ ]:
Result_PN_Sayonara["PN"].describe()

count    42.000000
mean     -0.688120
std       0.503518
min      -0.999997
25%      -0.983579
50%      -0.911561
75%      -0.657423
max       0.984270
Name: PN, dtype: float64

これは良そう通り思いっきりネガティブですね。

## 考察

極性辞書を用いて文章のネガポジ判定をすることが出来た。

しかしながら精度は低く、ネガティブ判定が強いように見える。

原因としてネガポジ判定する辞書が悪い、文脈を考慮していない。等の原因が考えられる。

独自関数を教材通りではなく多少アレンジして実装も試みた。可読性が向上したが、独自関数で分からない部分もあるので復習する事。

①　例えばこちらのlines = lines[:-2]としているが、上手く取り除かれていない事。

In [ ]:
import re
mecab = MeCab.Tagger('-Owakati')                     # 分かち書きに指定
def get_diclist(file):
    parsed = mecab.parse(file)                       # 分かち書きを行います。
    # 解析結果を改行ごとに区切ります
    lines = parsed.split(' ')                        # 文章を" " 空行で区切っています。
    lines = lines[:-2]                               # これが機能していない？
    #解析結果のリストを作成します
    diclist = []                                    # 解析結果はここに格納されます。
    for word in lines:
        datalist = {'BaseForm':word}                # 分かち書き結果をpick upして"BaseForm"というkey名を付けてvalueとして格納。
        diclist.append(datalist)                    # 追加
    return(diclist)


こちらの部分の解釈など。　`if baseword in PN_dict:`

In [ ]:
# 解析結果のリストからPNTableに存在する単語を抽出します

def add_pnvalue(old_wordlist):
    diclist_new = []
    new_pn_list = []                                                   # 入れ物の作成
    for word in old_wordlist:
        baseword = word['BaseForm']                                    # old_wordlistの分かち書きされた単語達。
        if baseword in PN_dict:                                        # in PN_dic⇒PN dict のkey 
            # PNTableに存在していれば極性値とその単語を追加します ⇒ floatにしている。 in dict python 
            pn = float(PN_dict[baseword])
        else:
            # 存在していなければnotfoundを明記します
            pn = 'not found'
        word['PN'] = pn                                                # 独自関数により新しく篩い分けられたpnを新たなPNのvalueとします。
        diclist_new.append(word)
    for d in diclist_new:                                              # ここのfor文で not found を間引いています。⇒ちょっと独自関数に無駄がないかな？
      if d['PN'] != 'not found':
        new_pn_list.append(d)   
    
    df = pd.DataFrame(new_pn_list)                                     # DFに変換しています。
    return(df)